# # MST

### # Graph and Other Data Structures

- load skt - DONE
- load dcs - DONE


- create the full graph - sktwseg_utf8 - DONE
- Nodelist - DONE
- Features-Adjacency Matrix - DONE
- Weight-Adjacency Matrix - DONE
- Conflict Matrix - or - dictionary (since it's sparse) - DONE


- new paths base on cng probability - DONE


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pickle
import numpy as np
from sentences import *
from DCS import *
from collections import defaultdict
import math
from IPython.display import display
import json
from word_definite import *
from romtoslp import *

In [3]:
loaded_SKT = pickle.load(open('../Simultaneous_CompatSKT_10K.p', 'rb'))
loaded_DCS = pickle.load(open('../Simultaneous_DCS_10K.p', 'rb'))

In [4]:
for k in list(loaded_SKT.keys())[100:]:
    loaded_SKT.pop(k, None)
    loaded_DCS.pop(k, None)

In [5]:
len(loaded_SKT)

100

In [6]:
mat_lem2cng_countonly = pickle.load(open('../NewData/ultimate_new_lastsem/mat_lem2cng_countonly.p', 'rb'))
mat_cng2lem_countonly = pickle.load(open('../NewData/ultimate_new_lastsem/mat_cng2lem_countonly.p', 'rb'))
mat_lem2cng_1D = pickle.load(open('../NewData/ultimate_new_lastsem/mat_lem2cng_1D.p', 'rb'))
mat_cng2lem_1D = pickle.load(open('../NewData/ultimate_new_lastsem/mat_cng2lem_1D.p', 'rb'))

### # Form Graph

In [44]:
def GetTrainingKit(sentenceObj, dcsObj):
    nodelist = GetNodes(sentenceObj)
    
    # Nodelist with only the correct_nodes
    nodelist2 = GetNodes(sentenceObj)
    nodelist2_to_correct_mapping = {}
    nodelist_correct = []
    search_key = 0
    first_key = 0
    for chunk_id in range(len(dcsObj.lemmas)):
        while nodelist2[first_key].chunk_id != chunk_id:
            first_key += 1
        for j in range(len(dcsObj.lemmas[chunk_id])):
            search_key = first_key

            while (nodelist2[search_key].lemma != rom_slp(dcsObj.lemmas[chunk_id][j])) or (nodelist2[search_key].cng != int(dcsObj.cng[chunk_id][j])):
                search_key += 1
    #         print((rom_slp(dcsObj.lemmas[chunk_id][j]), dcsObj.cng[chunk_id][j]))
    #         print(nodelist[search_key])
            nodelist2_to_correct_mapping[len(nodelist_correct)] = search_key
            nodelist_correct.append(nodelist2[search_key])
    return (nodelist, nodelist_correct, nodelist2_to_correct_mapping)

In [48]:
fn = 72
sentenceObj = loaded_SKT[list(loaded_SKT.keys())[fn]]
dcsObj = loaded_DCS[list(loaded_SKT.keys())[fn]]
# (chunkDict, lemmaList, wordList, revMap2Chunk, qu, cngList, verbs, tuplesMain, qc_pairs) = SentencePreprocess(sentenceObj)
(nodelist, nodelist_correct, nodelist_to_correct_mapping) = GetTrainingKit(sentenceObj, dcsObj)

In [49]:
nodelist_to_correct_mapping

{0: 0, 1: 1, 2: 4, 3: 5, 4: 20, 5: 10}

In [8]:
_edge_vector_dim = len(mat_cng2lem_1D)
_full_cnglist = list(mat_cng2lem_1D)
print(_edge_vector_dim)

276


In [9]:
import numpy as np
import math
from word_definite import *
from nnet import *
from heap_n_PrimMST import *
from word_definite import *

In [10]:
def organize_mst_nodes(mst_nodes):
    sol_lemmas = []
    sol_cngs = []
    print()
    total_tree_weight = 0
    for key, val in mst_nodes.items():
        print('{', end = '')
        for node in val:
            print(node.lemma, (node.cng, node.derived, node.src,), end = ', ')
            if node.src != -1:
                total_tree_weight += WScalarMat[node.src][node.id]
        print('};', end = ' ')
    print()
    print('Weight of the spanning tree: ', total_tree_weight)

In [67]:
neuralnet = NN(_edge_vector_dim, 200)

In [76]:
# Train
perm = np.random.permutation(100)[0:10]
print('Permutation: ', perm)
for iteriter in range(10):
    print('ITERATION ', iteriter)
    pickle.dump(neuralnet, open('outputs/neuralnet_trained.p', 'wb'))
    for fn in perm:
        sentenceObj = loaded_SKT[list(loaded_SKT.keys())[fn]]
        dcsObj = loaded_DCS[list(loaded_SKT.keys())[fn]]
        # (chunkDict, lemmaList, wordList, revMap2Chunk, qu, cngList, verbs, tuplesMain, qc_pairs) = SentencePreprocess(sentenceObj)
        try:
            (nodelist, nodelist_correct, nodelist_to_correct_mapping) = GetTrainingKit(sentenceObj, dcsObj)
        except IndexError:
            # print('Error with ', fn)
            continue

        conflicts_Dict = Get_Conflicts(nodelist)
        conflicts_Dict_correct = Get_Conflicts(nodelist_correct)

        featVMat = Get_Feat_Vec_Matrix(nodelist, conflicts_Dict)
        featVMat_correct = Get_Feat_Vec_Matrix(nodelist_correct, conflicts_Dict_correct)

        WScalarMat = Get_W_Scalar_Matrix_from_FeatVect_Matrix(featVMat, nodelist, conflicts_Dict, neuralnet)
        WScalarMat_correct = Get_W_Scalar_Matrix_from_FeatVect_Matrix(featVMat_correct, nodelist_correct, conflicts_Dict_correct, neuralnet)
        Total_Loss = 0

        # Get MST for the correct nodelist
        source = 0
        (mst_nodes_correct, mst_adj_graph_correct_0) = MST(nodelist_correct, WScalarMat_correct, conflicts_Dict_correct, source)
    #     print('Correct MST Score: ', GetMSTWeight(mst_nodes_correct, WScalarMat_correct))
        W_star = GetMSTWeight(mst_nodes_correct, WScalarMat_correct)

        # Convert correct spanning tree graph adj matrix to actual marix dimensions
        nodelen = len(nodelist)
        mst_adj_graph_correct = np.ndarray((nodelen, nodelen), np.bool)*False
        for i in range(mst_adj_graph_correct_0.shape[0]):
            for j in range(mst_adj_graph_correct_0.shape[1]):
                mst_adj_graph_correct[nodelist_to_correct_mapping[i], nodelist_to_correct_mapping[j]] = \
                mst_adj_graph_correct_0[i, j]

        # Get all MST
        dLdS = np.zeros(WScalarMat.shape)
        for source in range(len(nodelist)):
        #     print('\nSOURCE: {}'.format(source))
            (mst_nodes, mst_adj_graph) = MST(nodelist, WScalarMat, conflicts_Dict, source)    
        #     print('Correct MST Score for Souce {}: {}'.format(source, GetMSTWeight(mst_nodes, WScalarMat)))
            print('.', end = '')
            Total_Loss += (W_star - GetMSTWeight(mst_nodes, WScalarMat))
            dLdS_inner = 1 / WScalarMat
            dLdS_inner[mst_adj_graph_correct == 1] *= -1
            dLdS_inner = dLdS_inner*(mst_adj_graph^mst_adj_graph_correct)
            dLdS += dLdS_inner        
        neuralnet.Back_Prop(dLdS/len(nodelist), len(nodelist), featVMat)

        Total_Loss /= len(nodelist)
        print("\nFileKey: %d, Loss: %6.3f, Original MSTScore: %6.3f" % (fn, Total_Loss, W_star))

Permutation:  [52 77 73 40 86 39 57 27 91 53]
ITERATION  0
................
FileKey: 52, Loss:  0.930, Original MSTScore:  7.726
..................................
FileKey: 77, Loss:  0.042, Original MSTScore:  2.130
...............................................................
FileKey: 40, Loss: -3.499, Original MSTScore:  4.152
.....................................
FileKey: 86, Loss: -2.106, Original MSTScore:  7.731
..
FileKey: 39, Loss: -0.002, Original MSTScore:  1.001
.........................................
FileKey: 57, Loss: -1.871, Original MSTScore:  7.373


KeyboardInterrupt: 

In [72]:
# Test
minScore = np.inf
minMst = None

perm = np.random.permutation(100)[0:10]
print('Permutation: ', perm)
for fn in perm:
    sentenceObj = loaded_SKT[list(loaded_SKT.keys())[fn]]
    dcsObj = loaded_DCS[list(loaded_SKT.keys())[fn]]
    # (chunkDict, lemmaList, wordList, revMap2Chunk, qu, cngList, verbs, tuplesMain, qc_pairs) = SentencePreprocess(sentenceObj)
    try:
        (nodelist, nodelist_correct, _) = GetTrainingKit(sentenceObj, dcsObj)
    except IndexError:
        print('\x1b[31mError with {} \x1b[0m'.format(fn))
        continue
        
    conflicts_Dict = Get_Conflicts(nodelist)
    conflicts_Dict_correct = Get_Conflicts(nodelist_correct)
    
    featVMat = Get_Feat_Vec_Matrix(nodelist, conflicts_Dict)
    featVMat_correct = Get_Feat_Vec_Matrix(nodelist_correct, conflicts_Dict_correct)
    
    WScalarMat = Get_W_Scalar_Matrix_from_FeatVect_Matrix(featVMat, nodelist, conflicts_Dict, neuralnet)

    # Get all MST
    for source in range(len(nodelist)):
        (mst_nodes, mst_adj_graph) = MST(nodelist, WScalarMat, conflicts_Dict, source)
        print('.', end = '')
        score = GetMSTWeight(mst_nodes, WScalarMat)
        if(score < minScore):
            print('.', end = '')
            minScore = score
            minMst = mst_nodes
    dcsLemmas = [[rom_slp(l) for l in arr]for arr in dcsObj.lemmas]
    full_match = 0
    partial_match = 0
    for chunk_id, wdSplit in mst_nodes.items():
        for wd in wdSplit:
            # Match lemma
            search_result = [i for i, j in enumerate(dcsLemmas[chunk_id]) if j == wd.lemma]
            if len(search_result) > 0:
                partial_match += 1
            # Match CNG
            for i in search_result:
                if(dcsObj.cng[chunk_id][i] == str(wd.cng)):
                    full_match += 1
                    # print(wd.lemma, wd.cng)
                    break
    dcsLemmas = [l for arr in dcsObj.lemmas for l in arr]
    print('\nFull Match: {}, Partial Match: {}, OutOf {}, NodeCount: {}, '.\
          format(full_match, partial_match, len(dcsLemmas), len(nodelist)))

Permutation:  [22 86 26 32 96 41  3 13 15  0]
...................................................
Full Match: 0, Partial Match: 1, OutOf 6, NodeCount: 47, 
.....................................
Full Match: 2, Partial Match: 3, OutOf 7, NodeCount: 37, 
....................
Full Match: 3, Partial Match: 4, OutOf 6, NodeCount: 16, 
..............................................
Full Match: 3, Partial Match: 5, OutOf 7, NodeCount: 46, 
......
Full Match: 2, Partial Match: 2, OutOf 2, NodeCount: 4, 
................
Full Match: 5, Partial Match: 6, OutOf 6, NodeCount: 16, 
..............................................................................
Full Match: 3, Partial Match: 3, OutOf 6, NodeCount: 78, 
Error with 13 
..............................
Full Match: 5, Partial Match: 6, OutOf 8, NodeCount: 30, 
Error with 0 


Full Match: 2, Partial Match: 5


In [43]:
display(mst_nodes)
SeeDCS(dcsObj)

{0: [WD_Node[C: 0, P: 0, pA @(-38) => pibata]],
 1: [WD_Node[C: 1, P: 0, BAgavata @(31) => BAgavatam]],
 2: [WD_Node[C: 2, P: 0, rasa @(69) => rasam]],
 3: [WD_Node[C: 3, P: 0, Alaya @(71) => Alayam]]}

DCS ANALYZE
---------------
pibata BAgavataM rasam Alayam   
[['pā'], ['bhāgavata'], ['rasa'], ['ālaya']]
Lemmas: ['pA', 'BAgavata', 'rasa', 'Alaya']
[['-38'], ['71'], ['69'], ['31']]



In [20]:
# Use 37171.p
print(sentenceObj.sent_id)
SeeSentence(sentenceObj)
nodelist

22899
SKT ANALYZE
---------------
pibata BAgavataM rasam Alayam   
Analyzing  pibata
0 :  pibata ['pA'] [{'verb': ['imp. [1] ac. pl. 2']}]
0 :  pA ['pA'] []
Analyzing  BAgavatam
0 :  BAgavatam ['BAgavata'] [{'noun': ['acc. sg. m.', 'acc. sg. n.', 'nom. sg. n.']}]
0 :  BAga ['BAga'] [{'compound': ['iic.']}]
0 :  BAs ['BAs'] [{'compound': ['iic.']}]
0 :  BAs ['BAs', 'BA'] [{'noun': ['nom. sg. f.']}, {'noun': ['acc. pl. f.', 'nom. pl. f.']}]
0 :  BA ['BA'] [{'noun': ['nom. sg. f.']}]
1 :  aga ['aga'] [{'compound': ['iic.']}]
1 :  a ['a'] [{'compound': ['iic.']}]
2 :  ga ['ga'] [{'compound': ['iic.']}]
2 :  ga ['ga'] [{'compound': ['iic.']}]
4 :  vatam ['van'] [{'noun': ['acc. sg. n.', 'nom. sg. n.', 'acc. sg. m.'], 'verbform': ['"SKTMW227.html#H_van"'], 'verb': [['pp.']]}]
Analyzing  rasam
0 :  rasam ['rasa'] [{'noun': ['acc. sg. m.']}]
Analyzing  Alayam
0 :  Alayam ['Alaya'] [{'noun': ['acc. sg. m.', 'acc. sg. n.', 'nom. sg. n.']}]



[WD_Node[C: 0, P: 0, pA @(-38) => pibata],
 WD_Node[C: 1, P: 0, BAgavata @(31) => BAgavatam],
 WD_Node[C: 1, P: 0, BAgavata @(69) => BAgavatam],
 WD_Node[C: 1, P: 0, BAgavata @(71) => BAgavatam],
 WD_Node[C: 1, P: 0, BAga @(3) => BAga],
 WD_Node[C: 1, P: 0, BAs @(3) => BAs],
 WD_Node[C: 1, P: 0, BAs @(30) => BAs],
 WD_Node[C: 1, P: 0, BA @(80) => BAs],
 WD_Node[C: 1, P: 0, BA @(40) => BAs],
 WD_Node[C: 1, P: 0, BA @(30) => BA],
 WD_Node[C: 1, P: 1, aga @(3) => aga],
 WD_Node[C: 1, P: 1, a @(3) => a],
 WD_Node[C: 1, P: 2, ga @(3) => ga],
 WD_Node[C: 1, P: 2, ga @(3) => ga],
 WD_Node[C: 1, P: 4, van @(-190) => vatam],
 WD_Node[C: 1, P: 4, van @(31) => vatam],
 WD_Node[C: 1, P: 4, van @(69) => vatam],
 WD_Node[C: 1, P: 4, van @(71) => vatam],
 WD_Node[C: 2, P: 0, rasa @(69) => rasam],
 WD_Node[C: 3, P: 0, Alaya @(31) => Alayam],
 WD_Node[C: 3, P: 0, Alaya @(69) => Alayam],
 WD_Node[C: 3, P: 0, Alaya @(71) => Alayam]]

In [160]:
Get_W_Scalar_Matrix_from_FeatVect_Matrix(featVMat_correct, nodelist_correct, conflicts_Dict_correct, neuralnet)

array([[        inf,  1.00000141,  0.99999987,  1.00002211,  1.00000411,
         1.00000562,  1.0000008 ],
       [ 1.00000494,         inf,  0.99999999,  1.00002792,  1.0000046 ,
         1.00000294,  1.00000123],
       [ 0.99999763,  0.99999997,         inf,  1.00000312,  1.00000036,
         1.00000108,  1.00000045],
       [ 1.00000537,  1.00000193,  1.00000004,         inf,  1.00000797,
         1.00000671,  1.00000124],
       [ 1.00000356,  1.00000114,  1.00000002,  1.00002847,         inf,
                inf,  1.000001  ],
       [ 1.00000655,  1.00000098,  1.00000007,  1.00003223,         inf,
                inf,  1.00000144],
       [ 1.00000488,  1.00000215,  1.00000016,  1.00003128,  1.0000071 ,
         1.00000757,         inf]])